In [9]:
from google import genai
from google.genai import types

with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

client = genai.Client(api_key=api_key)

sys_instruct="All prompts should be answered with an in depth paper that is about 4 pages, written in plain text and include sources"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["Write a paper on the topic AI and the problems it couses to plagiarism"],
)


text = response.text

write_file = open("output.txt", "w")

## The AI Plagiarism Paradox: Redefining Authorship and Academic Integrity in the Age of Artificial Intelligence

The rapid advancement of Artificial Intelligence (AI) has ushered in an era of unprecedented capabilities, impacting various sectors from healthcare to finance. However, this technological revolution also poses significant challenges, particularly in the realm of education and academic integrity. One of the most pressing concerns is the increasing accessibility of AI-powered tools that can generate text, code, and even artistic creations, blurring the lines of authorship and raising complex questions about plagiarism. This paper will explore the multifaceted problems that AI poses to plagiarism detection and prevention, examining the evolving definition of authorship, the limitations of current detection methods, and potential solutions for maintaining academic integrity in the age of AI.

**The Shifting Sands of Authorship: Defining Original Work in an AI-Driven World**

T